## INTRODUCTION

The purpose of this project is to build a postgress databse with all the basic functionalities based on best design practices.

In [53]:
import psycopg2
conn = psycopg2.connect(dbname="crime_db")
# Creating a database cannot be part of a transaction block. herefore, to create a database, 
# we need to tell Postgres to immediately commit changes and not create transaction blocks. 
# We can do this by setting the connection.autocommit attribute of the connection to True
conn.autocommit=True

In [ ]:
cursor = conn.cursor()

cursor.execute("CREATE DATABASE crime_db1;")
conn.autocommit=False

In [43]:
conn = psycopg2.connect(dbname="crime_db1")

In [56]:
# create crimes schema
cursor.execute("CREATE SCHEMA crimes;")

### Obtaining column names: obtain the header row and assign it to the col_header & the first data row to first_row variables.


In [34]:
import csv
with open('boston.csv') as file:
    reader  = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

col_headers

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

In [35]:
first_row

['1',
 '619',
 'LARCENY ALL OTHERS',
 '2018-09-02',
 'Sunday',
 '42.35779134',
 '-71.13937053']

# Create a function for analyzing columns
Create a function to get unique column values. This function is helpful in the following ways:

* Checking whether an enumerated datatype might be a good choice for representing a column
* Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns


In [36]:
def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
        return values
    
for i in range(len(col_headers)):
    values = get_col_set('boston.csv', i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Analyzing maximum length of description column

In [37]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [38]:
offense_code = get_col_set('boston.csv', 1)
max_len = 0

for offense in offense_code:
    max_len = max(max_len, len(offense))

print(max_len)

4


In [39]:
description = get_col_set('boston.csv', 2)
max_len = 0

for description in description:
    max_len = max(max_len, len(description))

print(max_len)

58


## Creating a Table

* We'll create an enumerated data type for week day column since it has limited set of values (Monday to Sunday)
* Offense_code and description columns have 59 and 58 maximum lenght of values. For that reason, we'll limit the columns size to a datatype of Vachar(100).
* The lat and long columns holds lost ofprecision. On that note, we'll use the decimal data type
* The incident number will take an integer data type and will be used as the PRIMARY KEY
* Offense_code also takes an integer value
* Finally, we shall repressent the date column as a Date datatype

In [59]:

cursor.execute("""
            CREATE TYPE weekdayss AS ENUM('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');

""")
cursor.execute("""
    CREATE TABLE crimes.boston_crimes(
    incident_number INTEGER PRIMARY KEY,
    offense_code INTEGER,
    description VARCHAR(100),
    date DATE,
    day_of_the_week weekdayss,
    lat DECIMAL,
    long DECIMAL

);
""")


## Load the Data into the Table

The copy_expert function is the fastest method for loading a csv file into a table

In [62]:
# load data from boston.csv into the boston_crimes table in 'crimes' schema
with open("boston.csv") as f:
    cursor.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

#verify that the data is successfullly loaded

cursor.execute("SELECT * FROM crimes.boston_crimes;")
print(len(cursor.fetchall()))

298329


We revoke all previledges of the public group on the public schema so as to disbale user from inheriting previledges on that schema. By doing this, we don't need to revoke previledges on newly created users.

In [67]:
## Revoke Public Priviledges
cursor.execute("REVOKE ALL ON SCHEMA public FROM public;")

cursor.execute("REVOKE ALL ON DATABASE crime_db1 FROM public;")

## Creating Read-Only User Group
Let's create a read-write group with **NOLOGIN** since it's a group and not a user such that users under this group will not have automatic login access to tables.

We'll then grant SELECT previledge to the read-only user

We also grant CONNECT previledge to allow users under this group connct to the crime_db databse and use the crimes schema

In [ ]:
# create a readonly group with NOLOGIN
cursor.execute("CREATE GROUP readonly NOLOGIN;")
#  grant conect previledge on database to group
cursor.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
# grant usage on schema to group
cursor.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
# grant basic previledges on all tables in schema to group
cursor.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

## Creating Read-Write Group

We create a read-write group with NOLOGIN for the same reason as stated above.

This user however, has the previledges to connect to the crimes_db databse and alter the tables under the crimes schema 

In [71]:
# create a readonly group with NOLOGIN
cursor.execute("CREATE GROUP readwrite NOLOGIN;")
#  grant conect previledge on database to group
cursor.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
# grant usage on schema to group
cursor.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
# grant basic previledges on all tables in schema to group
cursor.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## Creating users for each group

We'll create a user named data_analyst with a password and assign into readonly group

& a data_scienctist user with a password and assign it ot readwrite group

In [72]:
cursor.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cursor.execute("GRANT readonly TO data_analyst;")

cursor.execute("CREATE USER data_scienctist WITH PASSWORD 'secret2';")
cursor.execute("GRANT readwrite TO data_scienctist;")

## Test The Database
Let's test our dartabase to verify that previledges are set accordingly as specified

In [101]:
#Let's close the connection & initiate a new connection
conn.close()

In [114]:
conn = psycopg2.connect(dbname='crime_db1', user='jay')
cursor = conn.cursor()

In [108]:
# Check users and groups.
cursor.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scienctist');
""")
for user in cursor:
    print(user)
print()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scienctist', False, False, False, True)



In [ ]:
cursor.execute("""
    SELECT *
    FROM information_schema.table_previledges
    WHERE grantee IN ('readonly', 'readwrite');
""")
               
for user in cursor:
    print(user)
conn.close()